In [1]:
from func.ultis import load_obj
import networkx as nx
import numpy as np

In [2]:
set_test = load_obj("ovules_training_graphs/graphs_dataset_train_with_augmentations_ovules_retrained_skript_testset")
print(len(set_test))

7


In [3]:
for graph in set_test:
    labels_dict = nx.get_node_attributes(graph, "label")
    labels = []
    for label in labels_dict.items():
        #print(label[1])
        labels.append(label[1])
    negatives = np.count_nonzero(labels)
    positives = len(labels) - negatives
    print(f"neg: {negatives}, \t pos: {positives}")


neg: 10971, 	 pos: 1164
neg: 8203, 	 pos: 4766
neg: 3065, 	 pos: 1527
neg: 7519, 	 pos: 630
neg: 10667, 	 pos: 425
neg: 4469, 	 pos: 503
neg: 10007, 	 pos: 1995


In [4]:
from func.graph_models import GCN
import torch

model = GCN(3, num_classes=1)
checkpoint_graph = 'output/graph_model_focal_ovules_retrained.pt'
model.load_state_dict(torch.load(checkpoint_graph, map_location=torch.device('cpu')))
model.eval()

Using backend: pytorch


GCN(
  (conv1): GraphConv(in=3, out=16, normalization=both, activation=None)
  (dropout): Dropout(p=0.5, inplace=False)
  (conv2): GraphConv(in=16, out=1, normalization=both, activation=None)
)

In [5]:
from func.graph_learning import VoxelGraphDataset
dataset = VoxelGraphDataset(set_test, with_ground_truth_labels=True)

/opt/anaconda3/envs/CellSeg3D_torch/lib/python3.8/site-packages/dgl/backend/pytorch/tensor.py:40: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  /Users/distiller/project/pytorch/torch/csrc/utils/tensor_new.cpp:210.)
  return th.as_tensor(data, dtype=dtype)


In [6]:
for voxel_graph in dataset:
    model_output = torch.sigmoid(model(voxel_graph, voxel_graph.ndata['feat']))
    predictions = (model_output > 0.5).type(torch.FloatTensor)
    
    labels = voxel_graph.ndata['label']
    
    accuracy = (predictions == labels).float().mean()
    print(accuracy)
    print("done")

tensor(0.8419)
done
tensor(0.6232)
done
tensor(0.6106)
done
tensor(0.8973)
done
tensor(0.9483)
done
tensor(0.8637)
done
tensor(0.8235)
done


In [11]:
print(set_test[0])

DiGraph with 54438 nodes and 1172568 edges
